<a href="https://colab.research.google.com/github/ryonce/Daily-Projects/blob/main/Logistic%2C_Random_Forest%2C_or_KNN_Core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - Your task is to create the best model to diagnose cancer. 

- Consider: Which is the most important metric for this task? 
  - I think recall is #1 as we do not want to risk any positives getting by us

- Which is the second most important? 
  - I think the #2 is Accuracy. We should be looking to be accurate in every case

In [4]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Models & Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Classification Metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, \
f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [8]:
# Import data

df = pd.read_csv('/content/wisconsinBreastCancer - wisconsinBreastCancer.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [10]:
# Turn target into values

df['diagnosis'] = df['diagnosis'].replace({'B':0, 'M':1})
df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [11]:
# Assign feature matrix and target vector

X = df.drop(columns = ['id', 'diagnosis'])
y = df['diagnosis']


# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Logistic Regression

In [12]:
# Make a linear regression instance

lr = LogisticRegression()

scaler = StandardScaler()

lr_pipe = make_pipeline(scaler, lr)

lr_pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

### Predictions and Scores

In [27]:
# Predictions

train_preds = lr_pipe.predict(X_train)
test_preds = lr_pipe.predict(X_test)

# Creating Recall Scores since #1 metric

lr_recall_train = recall_score(y_train, train_preds)
lr_recall_test = recall_score(y_test, test_preds)


print(f'Logistic Regression Train Recall Score: {round(lr_recall_train, 3)}')
print(f'Logistic Regression Test Recall Score: {round(lr_recall_test, 3)}')

Logistic Regression Train Recall Score: 0.975
Logistic Regression Test Recall Score: 0.981


In [28]:
lr_pipe.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('logisticregression', LogisticRegression())],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'logisticregression': LogisticRegression(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'logisticregression__C': 1.0,
 'logisticregression__class_weight': None,
 'logisticregression__dual': False,
 'logisticregression__fit_intercept': True,
 'logisticregression__intercept_scaling': 1,
 'logisticregression__l1_ratio': None,
 'logisticregression__max_iter': 100,
 'logisticregression__multi_class': 'auto',
 'logisticregression__n_jobs': None,
 'logisticregression__penalty': 'l2',
 'logisticregression__random_state': None,
 'logisticregression__solver': 'lbfgs',
 'logisticregression__tol': 0.0001,
 'logisticregression__verbose': 0,
 'logisticregression__warm_start': False}

### Best Params Find

In [29]:
# Logistic Regression

# Parameter grid

params = {
    'logisticregression__penalty' : ['l1','l2',], 
    'logisticregression__C'       : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'logisticregression__solver'  : ['liblinear'],
    'logisticregression__max_iter': [1000]}

In [30]:
# Create GridSearch Pipeline

lr_pipe_gs = GridSearchCV(lr_pipe, params)

lr_pipe_gs.fit(X_train, y_train)

lr_pipe_gs.best_params_

{'logisticregression__C': 1,
 'logisticregression__max_iter': 1000,
 'logisticregression__penalty': 'l2',
 'logisticregression__solver': 'liblinear'}

In [31]:
# Make a best linear regression instance

best_lr = LogisticRegression(max_iter = 1000, penalty = 'l2', C = 1, solver = 'liblinear')

best_lr_pipe = make_pipeline(scaler, best_lr)

best_lr_pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(C=1, max_iter=1000, solver='liblinear'))])

In [32]:
# Best Predictions

best_train_preds = best_lr_pipe.predict(X_train)
best_test_preds = best_lr_pipe.predict(X_test)

# Creating Recall Scores since #1 metric

best_lr_recall_train = recall_score(y_train, best_train_preds)
best_lr_recall_test = recall_score(y_test, best_test_preds)


print(f'Best Logistic Regression Train Recall Score: {round(best_lr_recall_train, 3)}')
print(f'Best Logistic Regression Test Recall Score: {round(best_lr_recall_test, 3)}')

Best Logistic Regression Train Recall Score: 0.975
Best Logistic Regression Test Recall Score: 0.981


# KNN Model

In [33]:
# Create an instance of the model
knn = KNeighborsClassifier()

knn_pipe = make_pipeline(scaler, knn)

knn_pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

### Predictions and Scores

In [34]:
# Predictions

knn_train_preds = knn_pipe.predict(X_train)
knn_test_preds = knn_pipe.predict(X_test)

# Creating Recall Scores since #1 metric

knn_recall_train = recall_score(y_train, knn_train_preds)
knn_recall_test = recall_score(y_test, knn_test_preds)

print(f'KNN Model Train Recall Score: {round(knn_recall_train, 3)}')
print(f'KNN Model Test Recall Score: {round(knn_recall_test, 3)}')

KNN Model Train Recall Score: 0.943
KNN Model Test Recall Score: 0.944


In [35]:
# Display the hyperparameters

knn_pipe.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('kneighborsclassifier', KNeighborsClassifier())],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'kneighborsclassifier': KNeighborsClassifier(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'kneighborsclassifier__algorithm': 'auto',
 'kneighborsclassifier__leaf_size': 30,
 'kneighborsclassifier__metric': 'minkowski',
 'kneighborsclassifier__metric_params': None,
 'kneighborsclassifier__n_jobs': None,
 'kneighborsclassifier__n_neighbors': 5,
 'kneighborsclassifier__p': 2,
 'kneighborsclassifier__weights': 'uniform'}

In [36]:
# Set the params

knn_params = {'kneighborsclassifier__n_neighbors': range(1,15),
          'kneighborsclassifier__weights':['uniform', 'distance']}

### Best Params Find

In [37]:
# Find the best params

knn_gs = GridSearchCV(knn_pipe, knn_params)
knn_gs.fit(X_train, y_train)
knn_gs.best_params_

{'kneighborsclassifier__n_neighbors': 5,
 'kneighborsclassifier__weights': 'uniform'}

## These are the same params I was using before so no tuning is necessary for recall score

# Random Forest Model

In [13]:
# Create an instance of the model

rf_tree = RandomForestClassifier()

rf_tree_pipe = make_pipeline(scaler, rf_tree)

rf_tree_pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

### Predictions and Scores

In [14]:
# Predictions

rf_tree_train_preds = rf_tree_pipe.predict(X_train)
rf_tree_test_preds = rf_tree_pipe.predict(X_test)

# Creating Recall Scores since #1 metric

rf_recall_train = recall_score(y_train, rf_tree_train_preds)
rf_recall_test = recall_score(y_test, rf_tree_test_preds)

print(f'Decision Tree Model Train Recall Score: {round(rf_recall_train, 3)}')
print(f'Decision Tree Model Test Recall Score: {round(rf_recall_test, 3)}')

Decision Tree Model Train Recall Score: 1.0
Decision Tree Model Test Recall Score: 0.944


In [15]:
# Display the hyperparameters

rf_tree_pipe.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('randomforestclassifier', RandomForestClassifier())],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'randomforestclassifier': RandomForestClassifier(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__ccp_alpha': 0.0,
 'randomforestclassifier__class_weight': None,
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__max_depth': None,
 'randomforestclassifier__max_features': 'sqrt',
 'randomforestclassifier__max_leaf_nodes': None,
 'randomforestclassifier__max_samples': None,
 'randomforestclassifier__min_impurity_decrease': 0.0,
 'randomforestclassifier__min_samples_leaf': 1,
 'randomforestclassifier__min_samples_split': 2,
 'randomforestclassifier__min_weight_fraction_leaf': 0.0,
 'randomforestclassifier__n_estimators': 100,
 'randomforestclassifier__n_jobs': None,
 

### Best Params Find

In [21]:
# Set the params

rf_tree_params = {'randomforestclassifier__max_depth': range(1,20,1), 
                   'randomforestclassifier__min_samples_leaf': [3,15,20],
}

In [22]:
# Find the best params

rf_tree_gs = GridSearchCV(rf_tree_pipe, rf_tree_params)
rf_tree_gs.fit(X_train, y_train)
rf_tree_gs.best_params_

{'randomforestclassifier__max_depth': 11,
 'randomforestclassifier__min_samples_leaf': 3}

In [23]:
# Create an instance of the best model

best_rf_tree = RandomForestClassifier(max_depth = 11, min_samples_leaf = 3)

best_rf_tree_pipe = make_pipeline(scaler, rf_tree)

best_rf_tree_pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [24]:
# Best Predictions

best_rf_tree_train_preds = best_rf_tree_pipe.predict(X_train)
best_rf_tree_test_preds = best_rf_tree_pipe.predict(X_test)

# Creating Best Recall Scores since #1 metric

best_rf_recall_train = recall_score(y_train, best_rf_tree_train_preds)
best_rf_recall_test = recall_score(y_test, best_rf_tree_test_preds)

print(f'Best Decision Tree Model Train Recall Score: {round(best_rf_recall_train, 3)}')
print(f'Best Decision Tree Model Test Recall Score: {round(best_rf_recall_test, 3)}')

Best Decision Tree Model Train Recall Score: 1.0
Best Decision Tree Model Test Recall Score: 0.944


# All Models Together

In [38]:
# All Scores

print(f'Best Logistic Regression Train Recall Score: {round(best_lr_recall_train, 3)}')
print(f'Best Logistic Regression Test Recall Score: {round(best_lr_recall_test, 3)}')

print('\n')

print(f'KNN Model Train Recall Score: {round(knn_recall_train, 3)}')
print(f'KNN Model Test Recall Score: {round(knn_recall_test, 3)}')

print('\n')

print(f'Best Decision Tree Model Train Recall Score: {round(best_rf_recall_train, 3)}')
print(f'Best Decision Tree Model Test Recall Score: {round(best_rf_recall_test, 3)}')

Best Logistic Regression Train Recall Score: 0.975
Best Logistic Regression Test Recall Score: 0.981


KNN Model Train Recall Score: 0.943
KNN Model Test Recall Score: 0.944


Best Decision Tree Model Train Recall Score: 1.0
Best Decision Tree Model Test Recall Score: 0.944


# Questions

- Which hyperparameters did you tune for each of the models?
  - For Logistic: penalty, C, solver, max_iter
  - For KNN: n_neighbors, weights
  - For Decision Tree: max_depth, min_samples_leaf

- Which model would you put into production to diagnose malignant tumors?
  - I would choose the Logistic Regression as it had a 98% Recall Score and I think catching as positive cases is priorty #1

- How does consideration of the meaning of false positive and false negative errors affect how you determine what 'best' means in this case?
  - You need to consider which false is more impactful for your data. Which one would hurt a consumer more, your business more, the overall health of the target. 